In [52]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib
import os
import pymorphy2
import re
from copy import deepcopy
import numpy as np

RusLem = pymorphy2.MorphAnalyzer()

def bprint(l, sep = " "):
    print sep.join(l)

In [42]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)
 

#get list of sentences from html pages
def prepare_html_pages(html_dir):
    html_files = os.listdir(html_dir)
    sentences = []
    for filename in html_files:
        text = "" 
        with open(html_dir + filename, "r") as f:
            for line in f:
                text += line
                
        visible_text = text_from_html(text)
        sentences += visible_text.strip().split(".")
    
    return sentences


#sentences_normaliation
def normalization(sentences):
    for i, sent in enumerate(sentences):
        tokens = re.findall('[\w]+', sent.strip().lower(), re.U)
        sentences[i] = " ".join([RusLem.parse(token)[0].normal_form for token in tokens])
    return sentences


#creating forward index {sent_id: list of word} and token_dictionary
def create_forward_index(sentences):
    forward_index = {}
    token_list = set()
    for sent_id, sent in enumerate(sentences):
        forward_index[sent_id] = sent.strip().split()
        for token in sent.strip().split():
            token_list.add(token)
        
    return forward_index, list(token_list)


def calculate_idf(forward_index, token_list):
    cnt_sentences = len(forward_index)
    token_df = {token:0 for token in token_list}
    for tokens in forward_index.values():
        for token in tokens:
            token_df[token] += 1
    token_idf = {token:np.log(cnt_sentences / float(token_df[token])) for token in token_df}
    return token_idf

def vec_normalization(vector):
    return vector / (1.0 * np.sqrt(np.sum(vector**2)))



def cos_similarity(vec_1, vec_2):
     return np.sum(vec_1 * vec_2) / float(np.sqrt(np.sum(vec_1**2)) * np.sqrt(np.sum(vec_2**2)))
    
class Collection(object):
    def __init__(self, htmls_dir="./htmls/"):
        self.raw_sentences = prepare_html_pages(htmls_dir)
        self.norm_sentences = normalization(deepcopy(raw_sentences))
        self.forward_index, token_list = create_forward_index(norm_sentences)
        self.tokens_idf = calculate_idf(forward_index, token_list)
        
    def doc2vec(self, sentence, idf=False):
        """
        Create sentence normalized vector
        """
        vector = [sentence.count(token) for token in self.token_list]
        if idf:
            vector = [np.log(1 + c) * np.log(self.token_idf[self.token_list[i]])
                      for i, c in enumerate(vector)]
        return vector
        
    def sents2vectors(self)
        self.simple_vectors = {sent_idx:self.doc2vec(self.forward_index[sent_idx], idf=False)
                               for sent_idx in self.forward_index.keys()}
        
        self.idf_vectors = {sent_idx:self.doc2vec(self.forward_index[sent_idx], idf=True)
                               for sent_idx in self.forward_index.keys()}
        
    def ranking(self, query):
        query_vec_simple = self.doc2vec(query, idf=False)
        query_vec_idf = self.doc2vec(query, idf=True)

        rank_simple = {}
        rank_tfidf = {}
        for sent_idx in self.simple_vectors.keys():
            rank_simple[sent_idx] = cos_similarity(query_vec_simple,
                                                   self.simple_vectors[sent_idx])
            rank_tfidf[sent_idx] = cos_similarity(query_vec_idf,
                                                   self.idf_vectors[sent_idx])
            
        return rank_simple, rank_tfidf
            

def print_serp(rank, query, sent_cnt = 20):
    sorted_rank = sorted(rank, key = operator.itemgetter(1), reverse=True)
    
    print "Query:",
    bprint(query)
    for pair in sorted_rank:
        print 

In [39]:

norm_sentences = normalization(deepcopy(raw_sentences))

In [51]:
print len(norm_sentences)
bprint(norm_sentences[100:101])
bprint(raw_sentences[100:101])
print len(forward_index), len(token_list)
bprint(token_list[:10])
bprint(forward_index[100])
print tokens_idf[token_list[10]], token_list[10]
print tokens_idf[token_list[8]], token_list[8]

1169
в конец век после падение основной они часть быть реанимировать политический активность христианский и исламский фундаменталист римский понтифик и далай лама
 В конце века после падения основной их части, была реанимирована политическая активность христианских и исламских фундаменталистов, Римского понтифика и Далай-Ламы
1169 4738
фокс растение препарат физика калькулятор промышленный сутки социализовать список нормальный
в конец век после падение основной они часть быть реанимировать политический активность христианский и исламский фундаменталист римский понтифик и далай лама
7.06390396147 житель
4.17353220358 список


In [24]:
x = prepare_html_pages("./htmls/")
print len(x), x[1], x[100]
t = create_forward_index(x)
print len(t), bprint(t[10])

1169   Marc Okrand [mɑrk ˈoʊkrænd]; 3 июля  1948 ) — американский  лингвист , известен как создатель языков для народов фантастических миров кинематографа США ( клингонский , вулканский и атлантский языки)  В конце века после падения основной их части, была реанимирована политическая активность христианских и исламских фундаменталистов, Римского понтифика и Далай-Ламы
1169 Считается, что последний носитель языка мутсун умер в 1930 году
None
